### Find Original Distance (GAS-PIPELINES)

In [1]:
import pandas as pd
import numpy as np
import math
import json
import glob


import rasterio
import shapefile
from IPython.display import display

In [15]:
def deg2rad(deg):
    return (deg * (math.pi/180))

def get_distance(p, q):
    # lat1, lon1 = p
    # lat2, lon2 = q

    lon1, lat1 = p
    lon2, lat2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)
 

    points = []
    for feature in data['features']:
        points = feature['geometry']['coordinates']
        # print(len(feature['geometry']['coordinates']))
    
    points = np.array(points)

    distance = 0
    for index in range(0, 6):
        # print(points[index])

        # distance = 0
        # for k in range(len(points)):
        current_path_list = []
        for i in range(1, len(points[index])):
            p = points[index][i - 1]
            q = points[index][i]
        
            # print(p, q)
            current_path_list.append(p)
            
            #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
            distance += get_distance(p, q)
    return distance, current_path_list

In [16]:
path = 'raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=807951.0194687467, bottom=1255157.4422908756, right=958381.0194687467, top=1417487.4422908756)
807951.0194687467 1255157.4422908756 958381.0194687467 1417487.4422908756


In [17]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(16233, 15043)
(array([-999,    0,    1,    2,    3,    4,    5,    6,    7], dtype=int16), array([      12,        6,  7847992, 46208978,  7581852, 76733148,
       12501737, 47954316, 45364978]))


In [18]:
index = 0
files = glob.glob('OUTPUTS/main_road.geojson')
print(files)
for f in files:
    dist, current_path_list = find_distance(f)
    # name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms')
    # index = index + 1

['OUTPUTS/main_road.geojson']
Distance = 130.04511913874845 Kms


/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [23]:
MAIN_KILO = dist
print(MAIN_KILO)

road_distance = dist

130.04511913874845


#### Main-Road Analysis

In [24]:
files = glob.glob('OUTPUTS/main_road_prj.geojson')
# print(files)

current_path_list = []

for file_name in files:

    f = open(file_name)
    data = json.load(f)
 
    points = []
    for feature in data['features']:
        points = feature['geometry']['coordinates']
    
    points = np.array(points)

    distance = 0
    for index in range(1, 2):
        current_path_list = []
        for i in range(1, len(points[index])):
            p = points[index][i - 1]
        
            current_path_list.append(p)

/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [25]:
pixels_dict = {}
for i in range(len(current_path_list)):

    ext_i, ext_j = current_path_list[i]
    # print(ext_i, ext_j)

    pi, pj = (ext_i - left) / 10, ((ext_j - top) / 10) * -1
    pixels_dict[ext_i, ext_j] = [pi, pj]


In [26]:
classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

check_pixels = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}


total_pxs = len(current_path_list)
for i in range(len(current_path_list)):
    ext_i, ext_j = current_path_list[i]
    pi, pj = pixels_dict[ext_i, ext_j]

    pi, pj = int(pi), int(pj)
    cs = mask_data[pi][pj]

    check_pixels[cs] += 1
    check_pixels[8] += 1


df_dict_pixels = {}
for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

for key, value in check_pixels.items():
    check_pixels[key] = np.round(((value / total_pxs) * 100), 2)

df_dict_perc = {}
for key, value in check_pixels.items():
    df_dict_perc[classes[key]] = f'{check_pixels[key]}%'

df_dict_dist = {}
for key, value in check_pixels.items():
    df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * road_distance) / 100)} Kms'


list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
display(route_stats)


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,0 pxs,0 pxs,0 pxs,2 pxs,8 pxs,0 pxs,0 pxs,10 pxs
ROAD PERCENTAGE,0.0%,0.0%,0.0%,20.0%,80.0%,0.0%,0.0%,100.0%
ROAD KILOMETERS,0.0 Kms,0.0 Kms,0.0 Kms,26.01 Kms,104.04 Kms,0.0 Kms,0.0 Kms,130.05 Kms


#### Route-Optimization Analysis

In [27]:
namming_dict = {
    1: 'A* Chebyshev distance',
    2: 'A* Diagonal distance',
    3: 'A* Euclidean distance',
    4: 'A* Manhattan distance',
    5: 'A* Proposed',
    6: 'Dijkstra\'s algorithm',
}

paths_dict = {
    1: 'paths/a-star-chesb',
    2: 'paths/a-star-diag',
    3: 'paths/a-star-euc',
    4: 'paths/a-star-man',
    5: 'paths/a-star-new',
    6: 'paths/dijk-route',
}

shape_dict = {
    1: 'routes-shape/a-star-chesb',
    2: 'routes-shape/a-star-diag',
    3: 'routes-shape/a-star-euc',
    4: 'routes-shape/a-star-man',
    5: 'routes-shape/a-star-new',
    6: 'routes-shape/dijk-route',
}

In [28]:
kilometers = []

In [29]:
for p in range(1, 7):
    path = paths_dict[p]
    path_list = []

    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    # new_path_list = []
    # for i in range(1, len(path_list) - 1):
    #     if i % 2 == 0:
    #         new_path_list.append(path_list[i])
    # new_path_list.append(path_list[len(path_list) - 1])

    # path_list = new_path_list

    ordinates_dict = {}


    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        ordinates_dict[pi, pj] = [ext_i, ext_j]

    
    ordinates_dict
    ordinates_list = []
    for key, value in ordinates_dict.items():
        ordinates_list.append(value)
    
    road_lenght = 0
    index = 1
    for _ in ordinates_list[1:]:
        i, j = ordinates_list[index][0], ordinates_list[index][1]
        x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

        if (x - i == 10.0) and (y - j == -10.0):
            road_lenght += math.sqrt(2 * 100)
        else:
            road_lenght += 10 
    
    print(f'Road-length ALGO = [{namming_dict[p]}] = {road_lenght / 1000} kms')
    kilometers.append(road_lenght / 1000)

Road-length ALGO = [A* Chebyshev distance] = 279.22232575487436 kms
Road-length ALGO = [A* Diagonal distance] = 287.5944700441202 kms
Road-length ALGO = [A* Euclidean distance] = 289.8147853370452 kms
Road-length ALGO = [A* Manhattan distance] = 293.57659341295636 kms
Road-length ALGO = [A* Proposed] = 292.55835964804805 kms
Road-length ALGO = [Dijkstra's algorithm] = 257.18887845310917 kms


In [30]:
kilometers

[279.22232575487436,
 287.5944700441202,
 289.8147853370452,
 293.57659341295636,
 292.55835964804805,
 257.18887845310917]

In [31]:
for k in range(1, 7):

    path = paths_dict[k]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    pixels_dict = {}
    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        # ordinates_dict[pi, pj] = [ext_i, ext_j]
        pixels_dict[ext_i, ext_j] = [pi, pj]
    
    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

    check_pixels = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
    }

    df_dict_pixels = {}

    r = shapefile.Reader(f'OUTPUTS/{shape_dict[k]}/shapefiles/test/multipoint.shp')
    feature = r.shapeRecords()[0]
    first = feature.shape.__geo_interface__  

    total_pxs = len(first['coordinates'])
    for cord in first['coordinates']:
        pi, pj = pixels_dict[cord]
        cs = mask_data[pi][pj]

        check_pixels[cs] += 1
        check_pixels[8] += 1
    
    
    for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    print(str)

    print(f'ALGORITHM = [{namming_dict[k]}]')
    # print(f'PIXELS LULC {check_pixels}')
    # print(f'PIXELS LULC {df_dict_pixels}')

    for key, value in check_pixels.items():
        check_pixels[key] = np.round(((value / total_pxs) * 100), 2)
    
    # print(f'PERCENTAGE LULC {check_pixels}')

    # classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland'}
    df_dict_perc = {}
    for key, value in check_pixels.items():
        df_dict_perc[classes[key]] = f'{check_pixels[key]}%'
    
    # print(f'ROAD PERCENTAGE = {df_dict_perc}')

    current_road_distance = kilometers[k - 1]
    road_distance = MAIN_KILO
    # print(f'CURRENT DISTANCE = {road_distance}, OPTIMIZED DISTANCE = {kilometers[k - 1]}')

    df_dict_dist = {}
    recheck = 0
    for key, value in check_pixels.items():
        df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * current_road_distance) / 100)} Kms'
        recheck += (check_pixels[key] * road_distance) / 100
    
    # print(f'ROAD KILOMETERS = {df_dict_dist}')
    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
    route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])

    header = ['Water', 'Dense-forest', 'Sparse-forest', 'Barren-land', 'Urban-land', 'Farmland', 'Fallowland', 'Total']
    display(route_stats)

    print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
    
    print(str)

    with open('df.txt', 'a') as f:

        f.write(str)
        f.write('\n')
        dfAsString = route_stats.to_string(header = header, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
        f.write(dfAsString)
        f.write(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
        # print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
        f.write('\n')
        f.write(str)
            

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Chebyshev distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,78 pxs,826 pxs,87 pxs,12792 pxs,462 pxs,1831 pxs,3669 pxs,19745 pxs
ROAD PERCENTAGE,0.4%,4.18%,0.44%,64.79%,2.34%,9.27%,18.58%,100.0%
ROAD KILOMETERS,1.12 Kms,11.67 Kms,1.23 Kms,180.91 Kms,6.53 Kms,25.88 Kms,51.88 Kms,279.22 Kms


CURRENT DISTANCE = 130.0 Kms, OPTIMIZED DISTANCE = 279.22232575487436 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Diagonal distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,68 pxs,674 pxs,124 pxs,13771 pxs,518 pxs,1825 pxs,3357 pxs,20337 pxs
ROAD PERCENTAGE,0.33%,3.31%,0.61%,67.71%,2.55%,8.97%,16.51%,100.0%
ROAD KILOMETERS,0.95 Kms,9.52 Kms,1.75 Kms,194.73 Kms,7.33 Kms,25.8 Kms,47.48 Kms,287.59 Kms


CURRENT DISTANCE = 130.0 Kms, OPTIMIZED DISTANCE = 287.5944700441202 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Euclidean distance]


...

CURRENT DISTANCE = 130.0 Kms, OPTIMIZED DISTANCE = 293.57659341295636 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Proposed]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,161 pxs,1092 pxs,168 pxs,12805 pxs,1100 pxs,2442 pxs,2920 pxs,20688 pxs
ROAD PERCENTAGE,0.78%,5.28%,0.81%,61.9%,5.32%,11.8%,14.11%,100.0%
ROAD KILOMETERS,2.28 Kms,15.45 Kms,2.37 Kms,181.09 Kms,15.56 Kms,34.52 Kms,41.28 Kms,292.56 Kms


CURRENT DISTANCE = 130.0 Kms, OPTIMIZED DISTANCE = 292.55835964804805 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [Dijkstra's algorithm]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,10 pxs,22 pxs,4 pxs,16791 pxs,38 pxs,91 pxs,1231 pxs,18187 pxs
ROAD PERCENTAGE,0.05%,0.12%,0.02%,92.32%,0.21%,0.5%,6.77%,100.0%
ROAD KILOMETERS,0.13 Kms,0.31 Kms,0.05 Kms,237.44 Kms,0.54 Kms,1.29 Kms,17.41 Kms,257.19 Kms


CURRENT DISTANCE = 130.0 Kms, OPTIMIZED DISTANCE = 257.18887845310917 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
